In [29]:
import sqlite3
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
from geopy.distance import distance
import re
# from  keys  import  client_id, api_key
import folium

In [47]:
pd.set_option('display.max_rows', 200)

In [31]:
from cleaning_utils import *
%load_ext autoreload
%autoreload 2

In [9]:
blight = pd.read_csv('../data/Blight_Violations.csv')

/Users/kelvinarellano/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9,11,15,16,17,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
blight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496335 entries, 0 to 496334
Data columns (total 36 columns):
X                             306227 non-null float64
Y                             306227 non-null float64
ticket_id                     496335 non-null int64
ticket_number                 496335 non-null object
agency_name                   496335 non-null object
inspector_name                496335 non-null object
violator_name                 496333 non-null object
violation_street_number       496335 non-null int64
violation_street_name         496270 non-null object
violation_zip_code            154162 non-null object
violator_id                   496335 non-null int64
mailing_address_str_number    496329 non-null object
mailing_address_str_name      496325 non-null object
city                          493079 non-null object
state                         492616 non-null object
zip_code                      493076 non-null object
non_us_str_code               3259 non-nul

In [12]:
MCM = pd.read_csv('../data/Motor_City_Mapping,_Winter_2013-14_Certified_Results.csv')

/Users/kelvinarellano/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
MCM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379549 entries, 0 to 379548
Data columns (total 33 columns):
OBJECTID             379549 non-null int64
D3_SurveyParcelID    379549 non-null object
CityParcelID2013     379549 non-null object
AddCombo             379549 non-null object
Address_Number       379549 non-null object
Address_Direction    379549 non-null object
Street               379549 non-null object
Zip                  379549 non-null object
PCount               379549 non-null int64
Blext_ID             377209 non-null float64
User_ID              373229 non-null float64
Reviewer_ID          359568 non-null float64
Time_Surveyed        379549 non-null object
Updated_At           379549 non-null object
Status               373229 non-null float64
Photo_URL            373217 non-null object
Structure            373227 non-null object
Use_                 373223 non-null object
Condition            261355 non-null object
Occupancy            261202 non-null object
Needs_B

In [14]:
MCM['CityParcelID2013'] = [x[:7] for x in MCM.CityParcelID2013]
MCM['D3_SurveyParcelID'] = [x[:7] for x in MCM.D3_SurveyParcelID]

In [24]:

MCM.drop(['GeoID10_BG','NAME10_Block','GEOID10_Block','Notes','Photo_URL','Time_Surveyed','Reviewer_ID','User_ID','Blext_ID','PCount','OBJECTID'], axis=1, inplace = True)

KeyError: "['GeoID10_BG' 'NAME10_Block' 'GEOID10_Block' 'Notes' 'Photo_URL'\n 'Time_Surveyed' 'Reviewer_ID' 'User_ID' 'Blext_ID' 'PCount' 'OBJECTID'] not found in axis"

In [27]:
MCM.drop(['GEOID10_Tract','Updated_At'], axis=1, inplace = True)

In [32]:
blight = blight[blight.payment_status == 'PAID IN FULL']

In [33]:
columns_to_drop = [
                 
                 'ticket_number',
                 'agency_name',
                 'inspector_name',
                 'violator_name',
                
                 'violator_id',
                 'mailing_address_str_number',
                 'mailing_address_str_name',
                 'city',
                 'state',
                 'zip_code',
                 'non_us_str_code',
                 'country',
                 
                 'ticket_issued_time',
                 'hearing_date',
                 'hearing_time',
                 
                 'violation_code',
                 'violation_description',
                 'disposition',
                 'clean_up_cost',
                 'payment_amount',
                 
                 
                 'collection_status',
                 'violation_address',
                 
                 'oid',
                 'geom',
                 'fine_amount']

In [34]:
blight = blight.drop(columns=columns_to_drop)

In [35]:
blight.violation_date = pd.to_datetime(blight['violation_date']).dt.date
blight.judgment_date = pd.to_datetime(blight['judgment_date']).dt.date
blight.payment_date = pd.to_datetime(blight['payment_date']).dt.date

In [36]:
blight.rename({'Y':'lat', 'X':'lon', 'parcelno': 'parcel_id'}, axis=1, inplace=True)

In [37]:
blight['compliance'] = blight.apply(compliance, axis=1)

In [38]:
blight = blight.dropna(subset=['parcel_id'])

In [39]:
blight['parcel_id'] = blight.apply(parse_parcel,axis=1)


In [40]:
blight['coordinates'] = blight.apply(coord_bligth,axis=1)

In [41]:
crimes = pd.read_csv('../data/Reported_Major_Crimes_2011_to_2014.csv')

In [42]:
crimes['lat'] = crimes.LOCATION.map(extract_lat_crime)
crimes['lon'] = crimes.LOCATION.map(extract_long_crime)

In [43]:
crimes =  crimes[ (crimes['lat'] < 45) | (crimes['lon'] < 85) ]

In [44]:
crimes['coordinates'] = crimes.apply(coord_bligth, axis=1)

In [45]:
crimes.INCIDENTDATE = pd.to_datetime(crimes['INCIDENTDATE']).dt.date

In [46]:
crimes.head(150)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Crimes.csv'